In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform=custom_transform,
)

train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)

val_data = datasets.Flowers102(
    root="data",
    split="val",
    download=True,
    transform=transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ]),
)

val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=64, shuffle=False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 344862509/344862509 [00:24<00:00, 14328988.23it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 567684.18it/s]


100%|██████████| 14989/14989 [00:00<00:00, 14775187.46it/s]


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 28 * 28, 1024)
        self.bn4 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 102)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 256 * 28 * 28)

        x = F.relu(self.bn4(self.fc1(x)))
        x = F.relu(self.bn5(self.fc2(x)))
        x = self.fc3(x)

        return F.log_softmax(x, dim=1)

model = NeuralNetwork().to(device)


In [ ]:

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)


In [ ]:
# Training loop
def train_loop(dataloader, model, loss_fn, optimizer, device):
    model.train()
    running_loss = 0.0
    for X, y in dataloader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

# Validation loop
def val_loop(dataloader, model, loss_fn, device):
    model.eval()
    val_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    val_loss /= len(dataloader)
    correct /= len(dataloader.dataset)
    return val_loss, correct * 100


In [ ]:
# Training
epochs = 100
start = time.time()
best_val_loss = float('inf')
patience = 3
counter = 0

for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train_loop(train_dataloader, model, loss_fn, optimizer, device)
    val_loss, val_accuracy = val_loop(val_dataloader, model, loss_fn, device)

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping")
            break

end = time.time()
print(f"Training took: {(end - start) / 60:.2f} minutes!")

Epoch 1/100
Train Loss: 4.3666, Val Loss: 4.4486, Val Accuracy: 4.41
Epoch 2/100
Train Loss: 3.8393, Val Loss: 4.0783, Val Accuracy: 10.49
Epoch 3/100
Train Loss: 3.5165, Val Loss: 3.7547, Val Accuracy: 19.71
Epoch 4/100
Train Loss: 3.2505, Val Loss: 3.4615, Val Accuracy: 24.22
Epoch 5/100
Train Loss: 2.9913, Val Loss: 3.2516, Val Accuracy: 30.78
Epoch 6/100
Train Loss: 2.7889, Val Loss: 3.1409, Val Accuracy: 32.94
Epoch 7/100
Train Loss: 2.6018, Val Loss: 2.9808, Val Accuracy: 36.86
Epoch 8/100
Train Loss: 2.4413, Val Loss: 2.8897, Val Accuracy: 36.86
Epoch 9/100
Train Loss: 2.2670, Val Loss: 2.8730, Val Accuracy: 39.12
Epoch 10/100
Train Loss: 2.1267, Val Loss: 2.7545, Val Accuracy: 40.00
Epoch 11/100
Train Loss: 2.0167, Val Loss: 2.6685, Val Accuracy: 41.96
Epoch 12/100
Train Loss: 1.8853, Val Loss: 2.6894, Val Accuracy: 42.65
Epoch 13/100
Train Loss: 1.7741, Val Loss: 2.6178, Val Accuracy: 43.24
Epoch 14/100
Train Loss: 1.6593, Val Loss: 2.5276, Val Accuracy: 44.41
Epoch 15/100
Tra